In [1]:
from pyspark.sql import SparkSession, dataframe, Row
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql import HiveContext
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from pyspark.sql.functions import col,trim,ltrim,rtrim,when,regexp_replace

import os
import re 

In [2]:
#sessão do spark com o Hive
#spark = SparkSession.builder.master("local[*]")\
#    .enableHiveSupport()\
#    .getOrCreate()

In [21]:
#Iniciando o tratamento dos dados
df_clientes = spark.sql("select * from desafio_curso.tbl_clientes")

In [23]:
#Convertendo os tipos de dados
df_clientes = df_clientes.withColumn("address_number",col("address_number").cast(IntegerType()))\
        .withColumn("business_unit",col("business_unit").cast(IntegerType()))\
        .withColumn("customerkey",col("customerkey").cast(IntegerType()))\
        .withColumn("division",col("division").cast(IntegerType()))\
        .withColumn("region_code",col("region_code").cast(IntegerType()))

In [25]:
#Tratando as colunas vazias
df_clientes = df_clientes.withColumn('line_of_business', regexp_replace('line_of_business', '   ', 'Não informado'))

In [26]:
df_clientes.createOrReplaceTempView('tb_clientes')

In [28]:
df_divisao = spark.sql("select * from desafio_curso.tbl_divisao")

In [30]:
df_divisao = df_divisao.withColumn("division",col("division").cast(IntegerType()))

In [31]:
df_divisao.createOrReplaceTempView('tb_divisao')

In [32]:
df_endereco = spark.sql("select * from desafio_curso.tbl_endereco")

In [33]:
df_endereco.printSchema()

root
 |-- address_number: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- customer_address_1: string (nullable = true)
 |-- customer_address_2: string (nullable = true)
 |-- customer_address_3: string (nullable = true)
 |-- customer_address_4: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- dt_foto: string (nullable = true)



In [6]:
df_regiao = spark.sql("select * from desafio_curso.tbl_regiao")

In [7]:
df_vendas = spark.sql("select * from desafio_curso.tbl_vendas")